In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sys
sys.path.append("..")
import neural_utils as neural
import pandas as pd
import scipy.constants as const


# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)
import matplotlib.pyplot as plt
from scipy.integrate import simpson

params = {"axes.labelsize": 14,
          "axes.titlesize": 16,}
plt.rcParams["axes.linewidth"] = 1
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update(params)

def place(ax):
  ax.tick_params(direction="in", which="minor", length=3)
  ax.tick_params(direction="in", which="major", length=5, labelsize=13)
  ax.grid(which="major", ls="dashed", dashes=(1, 3), lw=0.8, zorder=0)
  #ax.legend(frameon=True, loc="best", fontsize=12,edgecolor="black")
  fig.tight_layout()
  
  
def get_sim_data(filepath):
  data = pd.read_csv(filepath, skiprows=1, header=None, sep=r'\s+').to_numpy()
  rho_sim = data[:,0]
  P_sim = data[:,1]
  P_err_sim = data[:,2]
  return rho_sim, P_sim, P_err_sim


In [ ]:

def betaFexc_twotype(model_H, model_O, rho_H, rho_O, dx=0.03):
    """
    Calculate the excess free energy Fexc for a given density profile with functional line integration.

    model: The neural correlation functional
    rho: The density profile
    dx: The discretization of the input layer of the model
    """
    alphas = np.linspace(0, 1, 30)
    integrands = np.empty_like(alphas)
    for i, alpha in enumerate(alphas):
        
        c1H, c1O = neural.c1_twotype(model_H, model_O, alpha * rho_H, alpha * rho_O, input_bins=667, dx=0.03, return_c2=False) 
        
        integrands[i] = np.sum(rho_H * c1H + rho_O * c1O) * dx 
    Fexc = -simpson(integrands, x=alphas)
    return Fexc



In [ ]:
kB  = const.Boltzmann
angtrom3_to_m3 = 1e30
pa_to_atm = 101325


def get_bulk_thermodynamics_SR(ratio_H, ratio_O, rho_range, T, model_H, model_O):
    """
    Calculate the bulk thermodynamics of a two-component system using the neural network models.
    ratio_H: The ratio of the first component
    ratio_O: The ratio of the second component
    rho_range: The range of total densities to evaluate
    """

    Fexc_range = np.empty_like(rho_range)
    derivPhiH = np.empty_like(rho_range)
    derivPhiO = np.empty_like(rho_range)
    z_range = np.ones(1)

    betamuH = np.empty_like(rho_range)
    betamuO = np.empty_like(rho_range)


    for i in range(len(rho_range)):
    
        rhoH_array = z_range * rho_range[i] * ratio_H 
        rhoO_array = z_range * rho_range[i] * ratio_O
    
        Fexc_range[i] = kB * T * betaFexc_twotype(model_H, model_O, rhoH_array, rhoO_array, dx=0.03)
        c1H, c1O, = neural.c1_twotype(model_H, model_O, rhoH_array, rhoO_array, input_bins=667, dx=0.03, return_c2=False)
        derivPhiH[i] = -np.mean(c1H) * kB * T
        derivPhiO[i] = -np.mean(c1O) * kB * T
    
        betamuH[i] = np.log(np.mean(rhoH_array)) - np.mean(c1H)
        betamuO[i] = np.log(np.mean(rhoO_array)) - np.mean(c1O)
        
        
    P_range = (derivPhiH + kB*T) * rho_range * ratio_H + (derivPhiO + kB*T) * rho_range * ratio_O  - Fexc_range/0.03
    P_range_atm = P_range * angtrom3_to_m3/pa_to_atm
    
    return P_range_atm, betamuH, betamuO

def mu_correction(q, kappa_inv, temp):
    """
    Calculate the chemical potential correction
    """
    prefactor = const.elementary_charge**2 /(4 * const.pi * const.epsilon_0 * 1e-10 )
    beta = 1/ (const.Boltzmann * temp) 
    mid = - q**2 / (kappa_inv * np.pi**0.5  )
    return prefactor*mid*beta


def LR_pressure_correction(kappa_inv, temp):
    '''
    sigma: unit AA
    T: unit K
  
    Return correction in atm
    '''
    return (- kB*temp/(2 * np.pi**1.5 * kappa_inv**3) ) * angtrom3_to_m3/pa_to_atm

def get_mu_LR(density, ratio_H, ratio_O, q_H, q_O, temp, kappa_inv, model_H, model_O):
    """
    Calculate the chemical potential for a given density using the neural network models.
    density: The density to evaluate
    """

    rhoH_array = np.array([1]) * density * ratio_H
    rhoO_array = np.array([1]) * density * ratio_O
    c1H, c1O, = neural.c1_twotype(model_H, model_O, rhoH_array, rhoO_array, input_bins=667, dx=0.03, return_c2=False)
    mu_H = np.log(np.mean(rhoH_array)) - np.mean(c1H)
    mu_O = np.log(np.mean(rhoO_array)) - np.mean(c1O)
    return mu_H + mu_correction(q_H, kappa_inv, temp), mu_O + mu_correction(q_O, kappa_inv, temp)

## RPM model

In [ ]:
rhotot_range = np.linspace(1e-3, 0.04, 50)
ratio_H, ratio_O = 0.5, 0.5
q_H, q_O = 1.0, -1.0
temp = 4000

kappa_inv = 5.0
model_H_path = "../../models/RPM_H.keras"
model_O_path = "../../models/RPM_O.keras"
model_H = keras.models.load_model(model_H_path)
model_O = keras.models.load_model(model_O_path)

P_range_atm, betamuH, betamuO = get_bulk_thermodynamics_SR(ratio_H, ratio_O, rhotot_range, temp, model_H, model_O)
rho_sim, P_sim, P_err_sim = get_sim_data("../../data/bulk_sim_pressure/pressure_lr_RPM.txt")

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(8,3))
ax[0].plot(rhotot_range, P_range_atm + LR_pressure_correction(kappa_inv, temp), lw=1, color="crimson")
ax[0].fill_between(rho_sim, P_sim-P_err_sim, P_sim+P_err_sim, color="hotpink", alpha=0.3)
ax[0].set_xlabel(r"$\sum_{\nu}\rho_{\nu} $ [$\mathrm{\AA}^{-3}$]")
ax[0].set_ylabel(r"$P$ [atm]")


ax[1].plot(betamuH, rhotot_range*ratio_H, lw=1, color="crimson", label=r"H (SR)", ls='-')
ax[1].plot(betamuO, rhotot_range*ratio_O, lw=1, color="crimson", ls='--', label=r"O (SR)")
ax[1].plot(betamuO, rhotot_range*ratio_O, lw=2, color="crimson", ls=':')
ax[1].plot(betamuH+mu_correction(q_H, kappa_inv, temp), rhotot_range*ratio_H, lw=1, color="blue", ls='-', label=r"H (LR)")
ax[1].plot(betamuO+mu_correction(q_O, kappa_inv, temp), rhotot_range*ratio_O, lw=1, color="blue", ls='--', label=r"O (LR)")
ax[1].set_ylabel(r"$\rho_{\nu} $ [$\mathrm{\AA}^{-3}$]")
ax[1].set_xlabel(r"$\beta\mu_\nu$")
ax[1].legend(frameon=True, loc="best", fontsize=10,edgecolor="black")

place(ax[0])
place(ax[1])

In [ ]:
print(get_mu_LR(0.020, ratio_H, ratio_O, q_H, q_O, temp, kappa_inv, model_H, model_O))

## PM model

In [ ]:


rhotot_range = np.linspace(1e-3, 0.03, 50)
ratio_H, ratio_O = 0.5, 0.5
q_H, q_O = 1.0, -1.0
temp = 4000

kappa_inv = 5.0
model_H_path = "../../models/PM_H.keras"
model_O_path = "../../models/PM_O.keras"
model_H = keras.models.load_model(model_H_path)
model_O = keras.models.load_model(model_O_path)

P_range_atm, betamuH, betamuO = get_bulk_thermodynamics_SR(ratio_H, ratio_O, rhotot_range, temp, model_H, model_O)
rho_sim, P_sim, P_err_sim = get_sim_data("../../data/bulk_sim_pressure/pressure_lr_PM.txt")



In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(8,3))
ax[0].plot(rhotot_range, P_range_atm + LR_pressure_correction(kappa_inv, temp), lw=1, color="crimson")
ax[0].fill_between(rho_sim, P_sim-P_err_sim, P_sim+P_err_sim, color="hotpink", alpha=0.3)
ax[0].set_xlabel(r"$\sum_{\nu}\rho_{\nu} $ [$\mathrm{\AA}^{-3}$]")
ax[0].set_ylabel(r"$P$ [atm]")


ax[1].plot(betamuH, rhotot_range*ratio_H, lw=1, color="crimson", label=r"H (SR)", ls='-')
ax[1].plot(betamuO, rhotot_range*ratio_O, lw=1, color="crimson", ls='--', label=r"O (SR)")

ax[1].plot(betamuH+mu_correction(q_H, kappa_inv, temp), rhotot_range*ratio_H, lw=1, color="blue", ls='-', label=r"H (LR)")
ax[1].plot(betamuO+mu_correction(q_O, kappa_inv, temp), rhotot_range*ratio_O, lw=1, color="blue", ls='--', label=r"O (LR)")
ax[1].set_ylabel(r"$\rho_{\nu} $ [$\mathrm{\AA}^{-3}$]")
ax[1].set_xlabel(r"$\beta\mu_\nu$")
ax[1].legend(frameon=True, loc="best", fontsize=10,edgecolor="black")

place(ax[0])
place(ax[1])

In [ ]:
print(get_mu_LR(0.020, ratio_H, ratio_O, q_H, q_O, temp, kappa_inv, model_H, model_O))

## 2:1 PM model

In [ ]:


rhotot_range = np.linspace(1e-3, 0.035, 50)
ratio_H, ratio_O = 2/3, 1/3
q_H, q_O = 1.0, -2.0
temp = 8000

kappa_inv = 5.0
model_H_path = "../../models/21PM_H.keras"
model_O_path = "../../models/21PM_O.keras"
model_H = keras.models.load_model(model_H_path)
model_O = keras.models.load_model(model_O_path)

P_range_atm, betamuH, betamuO = get_bulk_thermodynamics_SR(ratio_H, ratio_O, rhotot_range, temp, model_H, model_O)
rho_sim, P_sim, P_err_sim = get_sim_data("../../data/bulk_sim_pressure/pressure_lr_21PM.txt")



In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(8,3))
ax[0].plot(rhotot_range, P_range_atm + LR_pressure_correction(kappa_inv, temp), lw=1, color="crimson")
ax[0].fill_between(rho_sim, P_sim-P_err_sim, P_sim+P_err_sim, color="hotpink", alpha=0.3)
ax[0].set_xlabel(r"$\sum_{\nu}\rho_{\nu} $ [$\mathrm{\AA}^{-3}$]")
ax[0].set_ylabel(r"$P$ [atm]")


ax[1].plot(betamuH, rhotot_range*ratio_H, lw=1, color="crimson", label=r"H (SR)", ls='-')
ax[1].plot(betamuO, rhotot_range*ratio_O, lw=1, color="crimson", ls='--', label=r"O (SR)")

ax[1].plot(betamuH+mu_correction(q_H, kappa_inv, temp), rhotot_range*ratio_H, lw=1, color="blue", ls='-', label=r"H (LR)")
ax[1].plot(betamuO+mu_correction(q_O, kappa_inv, temp), rhotot_range*ratio_O, lw=1, color="blue", ls='--', label=r"O (LR)")
ax[1].set_ylabel(r"$\rho_{\nu} $ [$\mathrm{\AA}^{-3}$]")
ax[1].set_xlabel(r"$\beta\mu_\nu$")
ax[1].legend(frameon=True, loc="best", fontsize=10,edgecolor="black")

place(ax[0])
place(ax[1])

In [ ]:
print(get_mu_LR(0.020, ratio_H, ratio_O, q_H, q_O, temp, kappa_inv, model_H, model_O))